# Introducing Thanksgiving Dinner Data

In [1]:
import pandas as pd
data = pd.read_csv("thanksgiving.csv", encoding = "Latin-1")
print(data.head(3))

   RespondentID Do you celebrate Thanksgiving?  \
0    4337954960                            Yes   
1    4337951949                            Yes   
2    4337935621                            Yes   

  What is typically the main dish at your Thanksgiving dinner?  \
0                                             Turkey             
1                                             Turkey             
2                                             Turkey             

  What is typically the main dish at your Thanksgiving dinner? - Other (please specify)  \
0                                                NaN                                      
1                                                NaN                                      
2                                                NaN                                      

  How is the main dish typically cooked?  \
0                                  Baked   
1                                  Baked   
2                                Roas

In [2]:
col_names = data.columns
print(col_names)

Index(['RespondentID', 'Do you celebrate Thanksgiving?',
       'What is typically the main dish at your Thanksgiving dinner?',
       'What is typically the main dish at your Thanksgiving dinner? - Other (please specify)',
       'How is the main dish typically cooked?',
       'How is the main dish typically cooked? - Other (please specify)',
       'What kind of stuffing/dressing do you typically have?',
       'What kind of stuffing/dressing do you typically have? - Other (please specify)',
       'What type of cranberry saucedo you typically have?',
       'What type of cranberry saucedo you typically have? - Other (please specify)',
       'Do you typically have gravy?',
       'Which of these side dishes aretypically served at your Thanksgiving dinner? Please select all that apply. - Brussel sprouts',
       'Which of these side dishes aretypically served at your Thanksgiving dinner? Please select all that apply. - Carrots',
       'Which of these side dishes aretypically served

# Filtering Out Rows From A DataFrame

In [3]:
print(data['Do you celebrate Thanksgiving?'].value_counts())

Yes    980
No      78
Name: Do you celebrate Thanksgiving?, dtype: int64


In [4]:
tg_yes = data[data['Do you celebrate Thanksgiving?'] == 'Yes']
print(len(tg_yes))

980


# Using value_counts To Explore Main Dishes

In [5]:
print(data['What is typically the main dish at your Thanksgiving dinner?'].value_counts())

Turkey                    859
Other (please specify)     35
Ham/Pork                   29
Tofurkey                   20
Chicken                    12
Roast beef                 11
I don't know                5
Turducken                   3
Name: What is typically the main dish at your Thanksgiving dinner?, dtype: int64


In [6]:
tofurkey = data[data['What is typically the main dish at your Thanksgiving dinner?'] == 'Tofurkey']
gravy_tofurkey = tofurkey['Do you typically have gravy?']
print(gravy_tofurkey)

4      Yes
33     Yes
69      No
72      No
77     Yes
145    Yes
175    Yes
218     No
243    Yes
275     No
393    Yes
399    Yes
571    Yes
594    Yes
628     No
774     No
820     No
837    Yes
860     No
953    Yes
Name: Do you typically have gravy?, dtype: object


# Figuring Out What Pies People Eat

In [7]:
ate_pies = (pd.isnull(data["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Apple"])
&
pd.isnull(data["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pecan"])
 &
 pd.isnull(data["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pumpkin"])
)

ate_pies.value_counts()

False    876
True     182
dtype: int64

# Converting Age To Numeric

In [8]:
data["Age"].value_counts()

45 - 59    286
60+        264
30 - 44    259
18 - 29    216
Name: Age, dtype: int64

In [9]:
def to_int(x):
    if pd.isnull(x) == True:
        return None
    x = x.split(" ")[0]
    x = x.replace("+", "")
    return int(x)
    
data["int_age"] = data["Age"].apply(to_int)
data["int_age"].describe()

count    1025.000000
mean       39.383415
std        15.398493
min        18.000000
25%        30.000000
50%        45.000000
75%        60.000000
max        60.000000
Name: int_age, dtype: float64

### Findings

We took only the first value of each string (the lower bound). It's a row approximation.

In [10]:
data["How much total combined money did all members of your HOUSEHOLD earn last year?"].value_counts()

$25,000 to $49,999      180
Prefer not to answer    136
$50,000 to $74,999      135
$75,000 to $99,999      133
$100,000 to $124,999    111
$200,000 and up          80
$10,000 to $24,999       68
$0 to $9,999             66
$125,000 to $149,999     49
$150,000 to $174,999     40
$175,000 to $199,999     27
Name: How much total combined money did all members of your HOUSEHOLD earn last year?, dtype: int64

In [11]:
def str_to_int(x):
    if pd.isnull(x) == True:
        return None
    x = x.split(" ")[0]
    if x == "Prefer":
        return None
    x = x.replace("$", "")
    x = x.replace(",", "")
    return int(x)
data["int_income"] = data["How much total combined money did all members of your HOUSEHOLD earn last year?"].apply(str_to_int)
data["int_income"].describe()

count       889.000000
mean      74077.615298
std       59360.742902
min           0.000000
25%       25000.000000
50%       50000.000000
75%      100000.000000
max      200000.000000
Name: int_income, dtype: float64

### Findings

We took only the first value of each string (the lower bound). It's a row approximation. The average income is high and there is also a large standard deviation.

# Correlating Travel Distance And Income

In [12]:
l_inc = data[data['int_income'] < 150000]['How far will you travel for Thanksgiving?']
l_inc.value_counts()

Thanksgiving is happening at my home--I won't travel at all                         281
Thanksgiving is local--it will take place in the town I live in                     203
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    150
Thanksgiving is out of town and far away--I have to drive several hours or fly       55
Name: How far will you travel for Thanksgiving?, dtype: int64

In [13]:
h_inc = data[data['int_income'] > 150000]['How far will you travel for Thanksgiving?']
h_inc.value_counts()

Thanksgiving is happening at my home--I won't travel at all                         49
Thanksgiving is local--it will take place in the town I live in                     25
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    16
Thanksgiving is out of town and far away--I have to drive several hours or fly      12
Name: How far will you travel for Thanksgiving?, dtype: int64

### Findings

It appears that more people with a high income have thanksgiving at home than people with low income.

# Linking Friendship And Age

In [14]:
data.pivot_table(
    index = "Have you ever tried to meet up with hometown friends on Thanksgiving night?", 
    columns = 'Have you ever attended a "Friendsgiving?"', 
    values = "int_age")

"Have you ever attended a ""Friendsgiving?""",No,Yes
Have you ever tried to meet up with hometown friends on Thanksgiving night?,,
No,42.283702,37.010526
Yes,41.475410,33.976744


In [15]:
data.pivot_table(
    index = "Have you ever tried to meet up with hometown friends on Thanksgiving night?", 
    columns = 'Have you ever attended a "Friendsgiving?"', 
    values = "int_income")

"Have you ever attended a ""Friendsgiving?""",No,Yes
Have you ever tried to meet up with hometown friends on Thanksgiving night?,,
No,78914.549654,72894.736842
Yes,78750.000000,66019.736842


### Findings

It appears that people who are younger are more likely to attend a Friendsgiving, and try to meet up with friends on Thanksgiving.

# Figure Out The Most Common Dessert People Eat

In [16]:
dessert_count = {}
desserts = data.loc[: ,'Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Apple cobbler':'Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Peach cobbler']

for col in desserts.columns:
    name = col.split(' ')[-1]
    dessert_count[name] = pd.notnull(desserts[col]).value_counts()[True]
    
dessert_count

{'Blondies': 16,
 'Brownies': 128,
 'Cheesecake': 191,
 'Cookies': 204,
 'Fudge': 43,
 'cake': 72,
 'cobbler': 103,
 'cream': 266}

In [24]:
print(max(dessert_count), dessert_count[max(dessert_count)])

cream 266
